In [3]:
import tensorflow as tf

2024-01-28 11:30:34.889148: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 11:30:34.921168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 11:30:34.921197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 11:30:34.921918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 11:30:34.926662: I tensorflow/core/platform/cpu_feature_guar

In [4]:
import numpy as np
import torch

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and reshape
train_images = train_images / 255.0
test_images = test_images / 255.0
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

# Convert to PyTorch format [batch_size, channels, height, width]
train_images_pt = torch.tensor(train_images).permute(0, 3, 1, 2).float()
test_images_pt = torch.tensor(test_images).permute(0, 3, 1, 2).float()

# If additional normalization is required for your PyTorch model, apply it here
# For example, if you use transforms.Normalize((0.1307,), (0.3081,)) in PyTorch, apply similar normalization
mean, std = 0.1307, 0.3081
train_images_pt = (train_images_pt - mean) / std
test_images_pt = (test_images_pt - mean) / std


In [5]:
# Define the LeNet model in TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(84, activation='sigmoid'),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

# Compile the model
model_tf.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

# Train the model
history = model_tf.fit(train_images, train_labels, epochs=3, batch_size=32, validation_split=0.1)


2024-01-28 11:30:36.614562: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 11:30:36.615241: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 11:30:36.615327: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/3


2024-01-28 11:30:37.798508: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-01-28 11:30:37.897090: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-01-28 11:30:37.897099: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2024-01-28 11:30:37.897122: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-01-28 11:30:38.089713: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 11:30:38.163125: I external/local_xla/xla/service/service.cc:168] XLA service 0x7

1688/1688 [==============================] - 4s 2ms/step - loss: 0.8812 - accuracy: 0.7149 - val_loss: 0.2320 - val_accuracy: 0.9333
Epoch 2/3
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2228 - accuracy: 0.9325 - val_loss: 0.1445 - val_accuracy: 0.9580
Epoch 3/3
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1477 - accuracy: 0.9541 - val_loss: 0.0962 - val_accuracy: 0.9723


In [6]:
# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.1121 - accuracy: 0.9651 - 253ms/epoch - 809us/step

Test accuracy: 0.9650999903678894


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class LeNetPT(nn.Module):
    def __init__(self):
        super(LeNetPT, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, 6, 5)  # 1 input channel, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(16 *4 * 4,120) # 256 * 120
        self.fc2 = nn.Linear(120, 84)         # 120 inputs, 84 outputs
        self.fc3 = nn.Linear(84, 10)          # 84 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # TODO: figure out the resize, currently work on batch_size = 1
        batch_size = x.size(0)
        x = x.reshape(x.size(0),16,-1)  # 16 output channels
        x = np.transpose(x, (0,2,1)).reshape(batch_size,-1)
        #x = x.reshape(batch_size,-1)

        # Fully connected layers
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)  # No activation function here, will use CrossEntropyLoss later
        return x
    

model_pt = LeNetPT()

In [8]:
# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[6].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the third dense layer (fc3) from model_tf to model_pt
weights, biases = model_tf.layers[7].get_weights()
model_pt.fc3.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc3.bias = nn.Parameter(torch.from_numpy(biases))

In [9]:
# # Create a controlled input (e.g., an array of ones)
# controlled_input_tf = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
# controlled_input_pt = torch.from_numpy(controlled_input_tf).permute(0, 3, 1, 2)  # For PyTorch

# Select the image for TensorFlow
controlled_input_tf = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)

In [10]:
# Test PyTorch Basic Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)

output_tf = model_tf.predict(controlled_input_tf) 
print("TF Basic Model Output:", output_tf)
print("PT Basic Model Output:", output_pt.cpu().numpy())

1/1 [==============================] - 0s 107ms/step
TF Basic Model Output: [[-1.6124442  -1.3553426   2.4211276   2.095878   -2.457887   -2.1878283
  -8.359751    7.455803   -2.1234655   0.07171768]]
PT Basic Model Output: [[-1.6124436  -1.355343    2.4211273   2.095877   -2.4578862  -2.1878288
  -8.359749    7.4558034  -2.1234653   0.07171769]]


In [25]:
# Assuming your model expects a single-channel (grayscale) image with 28x28 resolution
dummy_input_shape = (1, 1, 28, 28)  # Batch size of 1, 1 channel, 28x28 pixels

# Create a dummy input tensor with random data
dummy_input = torch.randn(dummy_input_shape).float()


In [76]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming the TensorFlow MNIST data has already been loaded
# Convert test_images to PyTorch tensor and permute
test_images_pt = torch.tensor(test_images).permute(0, 3, 1, 2).float()

# Assuming test_labels are already loaded
test_dataset = TensorDataset(test_images_pt, torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model_pt, test_loader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 96.26%


## Layer 1

In [532]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)

    def forward(self, x):
        return torch.sigmoid(self.conv1(x))

model_pt_basic = BasicLeNetPT()

model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1))
])

In [533]:
import numpy as np
# Transfer weights for TensorFlow
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for PyTorch
with torch.no_grad():
    model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
    model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))


In [536]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)


In [537]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)

# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 14ms/step
TensorFlow Basic Model Output: [[[[0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   ...
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]]

  [[0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   ...
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]]

  [[0.3405072  0.37062928 0.6257144  0.33070144

## Layer 2

In [538]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid')
])

In [539]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[1].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))


In [540]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)


In [541]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 21ms/step
TensorFlow Basic Model Output: [[[[2.57586502e-02 6.63225278e-02 3.52370068e-02 ... 2.83964783e-01
    3.78392786e-01 1.71559360e-02]
   [3.18357944e-02 9.97148827e-02 3.43728028e-02 ... 4.42336261e-01
    4.03742552e-01 2.45979186e-02]
   [4.90795746e-02 1.39458686e-01 3.03143449e-02 ... 5.38973391e-01
    4.56746191e-01 3.97939608e-02]
   ...
   [4.77694720e-02 6.36636093e-02 3.90103497e-02 ... 3.89785856e-01
    2.98766255e-01 4.16699834e-02]
   [3.69360596e-02 6.63337857e-02 3.34584303e-02 ... 2.79973477e-01
    3.29799414e-01 2.47441996e-02]
   [3.33412029e-02 6.84268177e-02 3.02623659e-02 ... 2.24366948e-01
    3.36898059e-01 1.78801026e-02]]

  [[5.97592928e-02 1.66600794e-01 2.22794805e-02 ... 6.77564323e-01
    2.82137245e-01 2.98533738e-02]
   [2.17131585e-01 3.99079263e-01 1.07417926e-02 ... 8.41790795e-01
    1.99286669e-01 6.10251874e-02]
   [6.20349586e-01 6.13339841e-01 3.73514532e-03 ... 8.61602783e-01
    9.75781530e-

## AvgPool + Flatten

In [77]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
import torch.nn.functional as F

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # Flattening the tensor to the correct size
        print ('x:', x.shape)
        #x = x.permute(3,2,0,1)
        x = x.reshape(x.size(0),16, -1)  # Reshape to [batch_size, 16*4*4]
        x = np.transpose(x, (0,2,1))
        print ('new x:', x)
        x = x.reshape(1,-1)
        #x = x.reshape(1,-1)

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten()
])

In [78]:
a = np.array([[
         [[3.5723e-01, 4.5777e-02, 5.6642e-02, 4.2337e-01],
          [2.1288e-01, 5.0078e-01, 1.6339e-01, 2.9913e-01],
          [9.1992e-01, 8.7455e-01, 2.3724e-01, 6.8523e-01],
          [8.1138e-01, 1.9003e-01, 6.3758e-03, 1.2055e-01]],

         [[4.8308e-01, 3.1106e-01, 5.4154e-01, 1.1745e-01],
          [7.9991e-03, 1.9359e-02, 3.3432e-01, 2.2252e-01],
          [1.2991e-02, 1.6436e-02, 2.4286e-01, 1.6083e-01],
          [5.4464e-03, 1.3997e-01, 4.2180e-01, 1.8740e-01]],

         [[3.2170e-01, 2.7298e-01, 3.5326e-02, 7.6450e-01],
          [9.5109e-01, 4.1637e-01, 1.7059e-01, 9.9765e-01],
          [3.9638e-01, 4.9827e-03, 4.1401e-01, 9.7265e-01],
          [3.7518e-02, 2.6161e-04, 8.0465e-01, 9.4228e-01]],

         [[9.6234e-01, 6.3446e-01, 7.8497e-01, 9.9889e-01],
          [1.0425e-01, 2.9717e-01, 1.7846e-01, 9.6586e-01],
          [9.5296e-01, 8.5555e-01, 7.1479e-01, 9.9898e-01],
          [9.1078e-01, 3.7230e-01, 6.3753e-01, 7.8140e-01]],

         [[5.4403e-01, 4.5540e-01, 6.0395e-01, 9.8221e-01],
          [2.6451e-01, 5.5743e-01, 5.3958e-01, 6.8985e-01],
          [9.3700e-01, 9.5921e-01, 9.8059e-01, 9.9786e-01],
          [7.8612e-01, 4.2739e-01, 1.7482e-01, 3.6958e-01]],

         [[6.6020e-01, 9.9102e-01, 8.3870e-01, 3.3561e-01],
          [2.5659e-01, 1.8054e-03, 2.7322e-03, 5.8188e-02],
          [5.8673e-03, 9.5297e-03, 7.6039e-03, 7.4197e-02],
          [9.2970e-02, 2.6396e-01, 4.9441e-01, 4.9260e-01]],

         [[9.9502e-01, 7.5909e-01, 9.0809e-01, 9.5490e-01],
          [3.1568e-03, 3.8385e-02, 8.8143e-02, 1.9937e-01],
          [1.7792e-01, 8.2782e-01, 7.9667e-01, 6.2599e-01],
          [5.7518e-01, 8.6944e-01, 5.0264e-01, 3.9224e-01]],

         [[3.7335e-01, 5.2908e-01, 3.8880e-01, 1.5501e-01],
          [8.6459e-01, 4.9262e-01, 1.8903e-01, 7.1074e-01],
          [4.4185e-02, 1.1017e-02, 1.0341e-01, 2.1341e-01],
          [6.5215e-02, 1.9144e-01, 8.4518e-01, 7.6709e-01]],

         [[7.8185e-01, 5.3509e-01, 9.5149e-01, 8.1755e-01],
          [1.2340e-03, 1.0570e-01, 4.1592e-01, 1.9644e-01],
          [3.0448e-01, 9.1760e-01, 8.7980e-01, 3.5192e-01],
          [6.8204e-01, 8.5161e-01, 3.2074e-01, 8.7232e-02]],

         [[9.8841e-01, 8.7884e-01, 9.8269e-01, 7.6363e-01],
          [5.2841e-02, 3.8924e-02, 7.4685e-01, 1.8598e-01],
          [8.0211e-02, 6.9800e-01, 9.5531e-01, 2.6941e-01],
          [4.7938e-01, 9.8505e-01, 7.3865e-01, 1.8938e-01]],

         [[9.9835e-01, 9.6591e-01, 9.8631e-01, 9.1282e-01],
          [2.0500e-01, 2.2773e-02, 7.6318e-01, 4.7738e-01],
          [1.2892e-01, 6.3485e-01, 9.8153e-01, 4.7129e-01],
          [4.9836e-01, 9.7686e-01, 8.6751e-01, 4.1846e-01]],

         [[8.2326e-01, 8.1415e-01, 4.7680e-01, 7.0314e-01],
          [9.8579e-01, 9.8720e-01, 9.1869e-01, 5.0148e-01],
          [9.9050e-01, 9.9447e-01, 7.9113e-01, 4.7052e-01],
          [9.9750e-01, 9.9385e-01, 7.4088e-01, 5.6988e-01]],

         [[9.5119e-01, 8.7655e-01, 5.0090e-01, 9.9662e-01],
          [7.9699e-01, 1.8493e-01, 2.5544e-01, 9.4455e-01],
          [4.4179e-01, 3.3896e-02, 7.5910e-01, 9.4222e-01],
          [4.1240e-01, 1.8616e-01, 9.9598e-01, 7.0146e-01]],

         [[1.3385e-01, 3.2317e-03, 1.8571e-02, 5.8818e-03],
          [8.2051e-02, 5.2408e-01, 5.1842e-01, 5.3531e-03],
          [6.4661e-01, 9.6453e-01, 3.2730e-01, 3.9188e-02],
          [3.0446e-01, 4.3663e-01, 9.8731e-04, 5.5072e-03]],

         [[8.7332e-01, 4.1994e-01, 1.0308e-01, 6.9599e-02],
          [1.2927e-03, 3.6074e-04, 3.5211e-01, 1.5637e-02],
          [1.0480e-03, 1.9545e-01, 6.5611e-01, 9.4675e-03],
          [6.2039e-03, 6.5116e-01, 4.0032e-01, 1.3742e-03]],

         [[6.5252e-02, 2.9379e-02, 1.9423e-01, 1.0593e-02],
          [1.9242e-01, 8.0646e-01, 7.9053e-01, 1.0609e-02],
          [8.1746e-01, 9.8786e-01, 4.0663e-01, 1.6456e-02],
          [8.3385e-01, 9.3526e-01, 7.4338e-03, 8.6007e-02]]]])

In [79]:
b = a.reshape(1,16,-1)

In [80]:
np.transpose(b, (0,2,1))

array([[[3.5723e-01, 4.8308e-01, 3.2170e-01, 9.6234e-01, 5.4403e-01,
         6.6020e-01, 9.9502e-01, 3.7335e-01, 7.8185e-01, 9.8841e-01,
         9.9835e-01, 8.2326e-01, 9.5119e-01, 1.3385e-01, 8.7332e-01,
         6.5252e-02],
        [4.5777e-02, 3.1106e-01, 2.7298e-01, 6.3446e-01, 4.5540e-01,
         9.9102e-01, 7.5909e-01, 5.2908e-01, 5.3509e-01, 8.7884e-01,
         9.6591e-01, 8.1415e-01, 8.7655e-01, 3.2317e-03, 4.1994e-01,
         2.9379e-02],
        [5.6642e-02, 5.4154e-01, 3.5326e-02, 7.8497e-01, 6.0395e-01,
         8.3870e-01, 9.0809e-01, 3.8880e-01, 9.5149e-01, 9.8269e-01,
         9.8631e-01, 4.7680e-01, 5.0090e-01, 1.8571e-02, 1.0308e-01,
         1.9423e-01],
        [4.2337e-01, 1.1745e-01, 7.6450e-01, 9.9889e-01, 9.8221e-01,
         3.3561e-01, 9.5490e-01, 1.5501e-01, 8.1755e-01, 7.6363e-01,
         9.1282e-01, 7.0314e-01, 9.9662e-01, 5.8818e-03, 6.9599e-02,
         1.0593e-02],
        [2.1288e-01, 7.9991e-03, 9.5109e-01, 1.0425e-01, 2.6451e-01,
         2.5659

In [81]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))


In [82]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input).float().permute(0, 3, 1, 2)

In [83]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().numpy())


1/1 [==============================] - 0s 36ms/step
x: torch.Size([1, 16, 4, 4])
new x: tensor([[[3.5723e-01, 4.8308e-01, 3.2170e-01, 9.6234e-01, 5.4403e-01,
          6.6020e-01, 9.9502e-01, 3.7335e-01, 7.8185e-01, 9.8841e-01,
          9.9835e-01, 8.2326e-01, 9.5119e-01, 1.3385e-01, 8.7332e-01,
          6.5252e-02],
         [4.5777e-02, 3.1106e-01, 2.7298e-01, 6.3446e-01, 4.5540e-01,
          9.9102e-01, 7.5909e-01, 5.2908e-01, 5.3509e-01, 8.7884e-01,
          9.6591e-01, 8.1415e-01, 8.7655e-01, 3.2317e-03, 4.1994e-01,
          2.9379e-02],
         [5.6642e-02, 5.4154e-01, 3.5326e-02, 7.8497e-01, 6.0395e-01,
          8.3870e-01, 9.0809e-01, 3.8880e-01, 9.5149e-01, 9.8269e-01,
          9.8631e-01, 4.7680e-01, 5.0090e-01, 1.8571e-02, 1.0308e-01,
          1.9423e-01],
         [4.2337e-01, 1.1745e-01, 7.6450e-01, 9.9889e-01, 9.8221e-01,
          3.3561e-01, 9.5490e-01, 1.5501e-01, 8.1755e-01, 7.6363e-01,
          9.1282e-01, 7.0314e-01, 9.9662e-01, 5.8818e-03, 6.9599e-02,
   

: 

## Dense Layer 1

In [17]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
import torch.nn.functional as F

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(256, 120) # 120 * 256

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        x = x.reshape(x.size(0),-1).reshape(16,-1)  # Reshape to [batch_size, 16*4*4]
        x = np.transpose(x, (1,0))
        x = x.reshape(1,-1)
        
        x = F.sigmoid(self.fc1(x))

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid')
])

In [18]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[5].get_weights()
model_tf_basic.layers[5].set_weights([weights, biases])


# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
print ('w:',weights)
model_pt_basic.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1,0))))
print ('w_T:',model_pt_basic.fc1.weight.shape)
model_pt_basic.fc1.bias = nn.Parameter(torch.from_numpy(biases))



w: [[-0.09746441 -0.07356118  0.12461983 ... -0.20353669  0.20595847
  -0.17135966]
 [ 0.24493529  0.15121421 -0.2389823  ...  0.23655343 -0.41860393
   0.23539582]
 [-0.0257456  -0.12429156 -0.1322509  ... -0.44357076  0.09737598
  -0.14918238]
 ...
 [ 0.22231688 -0.01645891  0.00374787 ...  0.04033783  0.18552913
  -0.15830086]
 [ 0.00637587 -0.32641515  0.23852865 ...  0.1092146  -0.2395123
   0.6130949 ]
 [-0.01752851  0.07713001  0.03625944 ...  0.03375389  0.10262422
  -0.1099361 ]]
w_T: torch.Size([120, 256])


In [19]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)

NameError: name 'controlled_input_tf' is not defined

In [20]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().numpy())


1/1 [==============================] - 0s 28ms/step
TensorFlow Basic Model Output: [[8.18893015e-01 6.09264314e-01 1.66922715e-03 4.11255181e-01
  2.49907449e-02 2.99167298e-02 7.96839595e-01 5.64328372e-01
  3.26452255e-01 6.82837749e-03 8.49843144e-01 6.74841583e-01
  8.32838356e-01 1.35930413e-02 9.28429067e-01 2.79215068e-01
  1.82010308e-01 8.45037960e-03 9.92652118e-01 9.68585014e-01
  2.62810066e-02 1.11930847e-01 7.52481520e-01 9.07907069e-01
  2.90007144e-03 8.81928265e-01 8.32566202e-01 9.12400186e-01
  8.92612875e-01 1.09533019e-01 9.80841100e-01 7.19543278e-01
  9.26168680e-01 9.86669302e-01 5.43678820e-01 9.56499994e-01
  1.10772308e-02 4.64457721e-01 9.96092975e-01 4.27115505e-04
  3.59919071e-01 9.72848415e-01 8.75910699e-01 1.85742781e-01
  7.08810687e-02 7.54384637e-01 7.68655062e-01 2.86270827e-01
  2.97551183e-03 7.74625782e-03 8.78870487e-01 8.34008455e-01
  9.68998671e-03 9.33030024e-02 9.78342414e-01 9.95083451e-01
  3.98346394e-01 3.37340939e-03 1.96328619e-04 5.

## Dense Layer 2

In [462]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
import os

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(120, 256) # 256 * 120
        self.fc2 = nn.Linear(120,84)         # 120 inputs, 84 outputs

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)).permute(3,2,0,1) # Convolution -> Sigmoid -> Avg Pool

        # Flattening the tensor to the correct size
        x = x.reshape(x.size(0),-1).reshape(1,-1)  # Reshape to [batch_size, 16*4*4]
        
        # Print the shape of x before and after each layer
        print("Before fc1:", x.shape)
        
        x = F.sigmoid(self.fc1(x))
        
        print("After fc1:", x.shape)
        x = F.sigmoid(self.fc2(x))
        print("After fc2:", x.shape)

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(84, activation='sigmoid')
])

In [463]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[5].get_weights()
model_tf_basic.layers[5].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[6].get_weights()
model_tf_basic.layers[6].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt_basic.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1,0))))
model_pt_basic.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[6].get_weights()
model_pt_basic.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1,0))))
model_pt_basic.fc2.bias = nn.Parameter(torch.from_numpy(biases))

In [464]:
# Create a controlled input (e.g., an array of ones)
controlled_input = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
controlled_input_pt = torch.from_numpy(controlled_input).permute(0, 3, 1, 2)  # For PyTorch


In [466]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().numpy())

1/1 [==============================] - 0s 9ms/step
Before fc1: torch.Size([1, 256])
After fc1: torch.Size([1, 120])
After fc2: torch.Size([1, 84])
TensorFlow Basic Model Output: [[0.7160134  0.8385422  0.0253077  0.42892176 0.6184138  0.9969837
  0.09171487 0.01182156 0.04299246 0.4810178  0.9516474  0.09384901
  0.8192958  0.55231595 0.10896716 0.00647116 0.04556889 0.8126056
  0.82992846 0.5289272  0.894237   0.08265778 0.9708183  0.24809329
  0.79544413 0.00390048 0.02803985 0.7674697  0.04256167 0.06272971
  0.3221807  0.43452185 0.6216037  0.04312156 0.97908485 0.72737664
  0.9310374  0.14843705 0.9657128  0.7040428  0.14954226 0.7755475
  0.2979573  0.5029429  0.85015994 0.19273664 0.04996477 0.02114206
  0.9176138  0.9626188  0.39863586 0.9899365  0.97770965 0.0713995
  0.02932361 0.17433517 0.99833655 0.8545451  0.11654612 0.21318336
  0.83583117 0.02372023 0.0214206  0.978412   0.852728   0.12194854
  0.23547943 0.55145293 0.06977712 0.49735156 0.98175037 0.28126732
  0.968428